# Five Laws Cognitive Governance Validator Tutorial

This tutorial demonstrates SIM-ONE's Five Laws validation system for ensuring AI-generated content adheres to cognitive governance principles.

## What You'll Learn
- Validate AI responses against the Five Laws of Cognitive Governance
- Batch validate multiple responses and compare compliance
- Iteratively refine text based on validation recommendations

## The Five Laws of Cognitive Governance

1. **Architectural Intelligence** - Intelligence emerges from coordination, not brute computational force
2. **Cognitive Governance** - Governed, structured processes over unconstrained generation
3. **Truth Foundation** - Grounded in absolute truth principles, not probabilistic drift
4. **Energy Stewardship** - Computational efficiency and resource-aware design
5. **Deterministic Reliability** - Consistent, predictable outcomes across executions

## Validation Strictness Levels
- **Lenient**: 60% compliance threshold - Good for exploratory content
- **Moderate**: 70% compliance threshold - Standard for most use cases
- **Strict**: 85% compliance threshold - For production governance systems

In [ ]:
# Standard library imports
import sys
import json
from pathlib import Path
from typing import Dict, Any, Optional, List, Literal
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add SIM-ONE to Python path
SIMONE_ROOT = Path("../code").resolve()
sys.path.insert(0, str(SIMONE_ROOT))

# Import Five Laws validators directly
from mcp_server.protocols.governance.five_laws_validator.law1_architectural_intelligence import ArchitecturalIntelligenceProtocol
from mcp_server.protocols.governance.five_laws_validator.law2_cognitive_governance import CognitiveGovernanceProtocol
from mcp_server.protocols.governance.five_laws_validator.law3_truth_foundation import TruthFoundationProtocol
from mcp_server.protocols.governance.five_laws_validator.law4_energy_stewardship import EnergyStewardshipProtocol
from mcp_server.protocols.governance.five_laws_validator.law5_deterministic_reliability import DeterministicReliabilityProtocol

# Configure plotting
%matplotlib inline
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Setup complete")

In [ ]:
class FiveLawsValidator:
    """
    Unified validator for all Five Laws of Cognitive Governance.
    """

    def __init__(self):
        """Initialize all five law protocols."""
        self.law1 = ArchitecturalIntelligenceProtocol()
        self.law2 = CognitiveGovernanceProtocol()
        self.law3 = TruthFoundationProtocol()
        self.law4 = EnergyStewardshipProtocol()
        self.law5 = DeterministicReliabilityProtocol()

        # Strictness thresholds
        self.thresholds = {
            "lenient": 60.0,
            "moderate": 70.0,
            "strict": 85.0
        }

    def validate(
        self,
        text: str,
        strictness: Literal["lenient", "moderate", "strict"] = "moderate",
        context: Optional[Dict[str, Any]] = None
    ) -> Dict[str, Any]:
        """
        Validate text against all Five Laws.

        Args:
            text: Text to validate
            strictness: Validation threshold level
            context: Optional context for validation

        Returns:
            Dictionary with scores, violations, recommendations, and pass/fail status
        """
        if not text or not text.strip():
            raise ValueError("Text to validate cannot be empty")

        # Prepare validation context
        validation_context = context or {}
        validation_context["text"] = text

        # Validate against each law
        law1_result = self.law1.evaluate(validation_context)
        law2_result = self.law2.evaluate(validation_context)
        law3_result = self.law3.evaluate(validation_context)
        law4_result = self.law4.evaluate(validation_context)
        law5_result = self.law5.evaluate(validation_context)

        # Extract scores
        law1_score = law1_result.get("score", 0.0)
        law2_score = law2_result.get("score", 0.0)
        law3_score = law3_result.get("score", 0.0)
        law4_score = law4_result.get("score", 0.0)
        law5_score = law5_result.get("score", 0.0)

        # Calculate overall compliance
        overall_compliance = (law1_score + law2_score + law3_score + law4_score + law5_score) / 5.0

        # Collect violations and recommendations
        violations = []
        recommendations = []
        strengths = []

        for law_name, result in [
            ("Law 1 (Architectural Intelligence)", law1_result),
            ("Law 2 (Cognitive Governance)", law2_result),
            ("Law 3 (Truth Foundation)", law3_result),
            ("Law 4 (Energy Stewardship)", law4_result),
            ("Law 5 (Deterministic Reliability)", law5_result)
        ]:
            violations.extend(result.get("violations", []))
            recommendations.extend(result.get("recommendations", []))
            if result.get("score", 0) >= 80.0:
                strengths.append(f"{law_name}: Strong compliance")

        # Determine pass/fail status
        threshold = self.thresholds[strictness]
        if overall_compliance >= threshold:
            status = "PASS"
        elif overall_compliance >= threshold - 10:
            status = "CONDITIONAL"
        else:
            status = "FAIL"

        return {
            "scores": {
                "law1_architectural_intelligence": law1_score,
                "law2_cognitive_governance": law2_score,
                "law3_truth_foundation": law3_score,
                "law4_energy_stewardship": law4_score,
                "law5_deterministic_reliability": law5_score,
                "overall_compliance": overall_compliance
            },
            "pass_fail_status": status,
            "strictness_level": strictness,
            "threshold": threshold,
            "violations": violations,
            "recommendations": recommendations,
            "strengths": strengths
        }


# Initialize validator
validator = FiveLawsValidator()
print("✓ Five Laws Validator initialized")
print(f"  Available strictness levels: {list(validator.thresholds.keys())}")

## Example 1: Validate a Compliant Response

Let's validate a response that demonstrates good governance principles.

In [ ]:
text = """
To analyze this dataset, I will use a coordinated multi-protocol approach:

1. ESL protocol will assess emotional context
2. REP protocol will handle logical reasoning
3. Results will be validated against ground truth

This architecture leverages protocol specialization for efficient processing
while maintaining deterministic outcomes through structured workflows.
"""

result = validator.validate(text, strictness="moderate")

print(json.dumps(result, indent=2))

print(f"\n{'='*80}")
print(f"Overall Compliance: {result['scores']['overall_compliance']:.1f}%")
print(f"Status: {result['pass_fail_status']}")
print(f"Strictness: {result['strictness_level']} (≥{result['threshold']}%)")

if result["strengths"]:
    print(f"\n✓ Strengths:")
    for strength in result["strengths"]:
        print(f"  - {strength}")

if result["violations"]:
    print(f"\n✗ Violations ({len(result['violations'])})")
    for violation in result["violations"][:3]:
        print(f"  - {violation}")

## Example 2: Validate a Non-Compliant Response

In [ ]:
text = """
Just use the biggest model available and throw more compute at it.
The AI will figure it out eventually.
"""

result = validator.validate(text, strictness="moderate")

print(f"Overall Compliance: {result['scores']['overall_compliance']:.1f}%")
print(f"Status: {result['pass_fail_status']}\n")

print("Individual Law Scores:")
print(f"  Law 1 (Architectural): {result['scores']['law1_architectural_intelligence']:.1f}%")
print(f"  Law 2 (Governance): {result['scores']['law2_cognitive_governance']:.1f}%")
print(f"  Law 3 (Truth): {result['scores']['law3_truth_foundation']:.1f}%")
print(f"  Law 4 (Energy): {result['scores']['law4_energy_stewardship']:.1f}%")
print(f"  Law 5 (Reliability): {result['scores']['law5_deterministic_reliability']:.1f}%")

print(f"\nTop Recommendations:")
for rec in result["recommendations"][:5]:
    print(f"  • {rec}")

## Example 3-5: Various Strictness Levels

In [ ]:
# Example 3: Lenient validation
text = "We'll use AI to help with this task."

result_lenient = validator.validate(text, strictness="lenient")
print(f"Lenient (≥60%): {result_lenient['pass_fail_status']} - {result_lenient['scores']['overall_compliance']:.1f}%")

# Example 4: Moderate validation (same text)
result_moderate = validator.validate(text, strictness="moderate")
print(f"Moderate (≥70%): {result_moderate['pass_fail_status']} - {result_moderate['scores']['overall_compliance']:.1f}%")

# Example 5: Strict validation (same text)
result_strict = validator.validate(text, strictness="strict")
print(f"Strict (≥85%): {result_strict['pass_fail_status']} - {result_strict['scores']['overall_compliance']:.1f}%")

print(f"\n💡 Same text, different thresholds produce different pass/fail outcomes")

## Example 6: Output Format Options

The validator supports both structured JSON and human-readable summaries.

In [ ]:
text = "This approach uses protocol coordination for architectural intelligence."

# JSON format (default - already shown in previous examples)
result_json = validator.validate(text, strictness="moderate")

# Create human-readable summary
def format_summary(result: Dict[str, Any]) -> str:
    """Format validation result as human-readable summary."""
    summary = []
    summary.append("Five Laws Validation Report")
    summary.append("=" * 80)
    summary.append(f"\nOverall Compliance: {result['scores']['overall_compliance']:.1f}%")
    summary.append(f"Status: {result['pass_fail_status']}")
    summary.append(f"Strictness: {result['strictness_level']} (threshold: {result['threshold']}%)\n")

    summary.append("Individual Law Scores:")
    summary.append(f"  1. Architectural Intelligence:    {result['scores']['law1_architectural_intelligence']:.1f}%")
    summary.append(f"  2. Cognitive Governance:          {result['scores']['law2_cognitive_governance']:.1f}%")
    summary.append(f"  3. Truth Foundation:              {result['scores']['law3_truth_foundation']:.1f}%")
    summary.append(f"  4. Energy Stewardship:            {result['scores']['law4_energy_stewardship']:.1f}%")
    summary.append(f"  5. Deterministic Reliability:     {result['scores']['law5_deterministic_reliability']:.1f}%\n")

    if result.get("violations"):
        summary.append(f"Violations ({len(result['violations'])})")
        for v in result["violations"][:5]:
            summary.append(f"  - {v}")
        summary.append("")

    if result.get("recommendations"):
        summary.append(f"Recommendations ({len(result['recommendations'])})")
        for r in result["recommendations"][:5]:
            summary.append(f"  - {r}")

    return "\n".join(summary)

# Display summary format
print(format_summary(result_json))

## Example 7: Batch Validation - Compare Multiple Responses

Validate multiple AI responses and compare their compliance scores.

In [ ]:
# Multiple responses to validate
responses = [
    "Use a coordinated multi-protocol architecture for efficient processing.",
    "Just throw everything at a large language model.",
    "Apply ESL for emotion analysis, then REP for reasoning with validation.",
    "Try different approaches until something works.",
    "Implement deterministic workflows with protocol specialization."
]

# Validate all responses
results = []
for i, text in enumerate(responses, 1):
    result = validator.validate(text, strictness="moderate")
    results.append({
        "id": i,
        "text_preview": text[:50] + "..." if len(text) > 50 else text,
        "overall": result["scores"]["overall_compliance"],
        "law1": result["scores"]["law1_architectural_intelligence"],
        "law2": result["scores"]["law2_cognitive_governance"],
        "law3": result["scores"]["law3_truth_foundation"],
        "law4": result["scores"]["law4_energy_stewardship"],
        "law5": result["scores"]["law5_deterministic_reliability"],
        "status": result["pass_fail_status"],
        "violations": len(result["violations"]),
        "recommendations": len(result["recommendations"])
    })

# Create comparison table
df = pd.DataFrame(results)

print("Batch Validation Comparison")
print("=" * 120)
print(df.to_string(index=False))

# Summary statistics
print(f"\n📊 Summary:")
print(f"  Total responses: {len(responses)}")
print(f"  Passed: {(df['status'] == 'PASS').sum()}")
print(f"  Conditional: {(df['status'] == 'CONDITIONAL').sum()}")
print(f"  Failed: {(df['status'] == 'FAIL').sum()}")
print(f"  Average compliance: {df['overall'].mean():.1f}%")

# Visualize comparison
fig, ax = plt.subplots(figsize=(12, 6))

x = df['id']
width = 0.15
x_pos = range(len(x))

ax.bar([p - 2*width for p in x_pos], df['law1'], width, label='Law 1: Architectural', alpha=0.8)
ax.bar([p - width for p in x_pos], df['law2'], width, label='Law 2: Governance', alpha=0.8)
ax.bar(x_pos, df['law3'], width, label='Law 3: Truth', alpha=0.8)
ax.bar([p + width for p in x_pos], df['law4'], width, label='Law 4: Energy', alpha=0.8)
ax.bar([p + 2*width for p in x_pos], df['law5'], width, label='Law 5: Reliability', alpha=0.8)

ax.axhline(y=70, color='red', linestyle='--', alpha=0.5, label='Moderate Threshold (70%)')
ax.set_xlabel('Response ID', fontsize=12)
ax.set_ylabel('Compliance Score (%)', fontsize=12)
ax.set_title('Five Laws Compliance Comparison', fontsize=14)
ax.set_xticks(x_pos)
ax.set_xticklabels(df['id'])
ax.legend(loc='upper right')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig("batch_validation_comparison.png", dpi=150, bbox_inches='tight')
plt.show()

## Example 8: Iterative Validation with Refinement

Validate text iteratively, showing how to improve compliance over multiple rounds.

In [ ]:
def iterative_validate_with_refinement(
    text: str,
    max_iterations: int = 3,
    threshold: float = 80.0,
    strictness: Literal["lenient", "moderate", "strict"] = "moderate"
) -> Dict[str, Any]:
    """
    Validate text with full state tracking for AI-client orchestration workflow.
    
    This function validates text and returns comprehensive feedback for iterative
    refinement. Designed for AI clients (like Claude Code) to:
    1. Call this function with text to validate
    2. Receive validation scores and recommendations
    3. Revise text based on recommendations
    4. Call again with revised text until pass or max_iterations
    
    Args:
        text: Text to validate (initial or revised)
        max_iterations: Maximum validation attempts (default: 3)
        threshold: Pass threshold percentage (default: 80.0)
        strictness: Validation strictness level
    
    Returns:
        Complete validation results with scores, recommendations, and pass/fail status
    """
    # Validate the text
    result = validator.validate(text, strictness=strictness)
    
    # Extract key metrics
    overall_compliance = result["scores"]["overall_compliance"]
    status = result["pass_fail_status"]
    passed = overall_compliance >= threshold
    
    # Build comprehensive response
    response = {
        "text": text,
        "validation_result": {
            "overall_compliance": overall_compliance,
            "individual_scores": {
                "law1_architectural_intelligence": result["scores"]["law1_architectural_intelligence"],
                "law2_cognitive_governance": result["scores"]["law2_cognitive_governance"],
                "law3_truth_foundation": result["scores"]["law3_truth_foundation"],
                "law4_energy_stewardship": result["scores"]["law4_energy_stewardship"],
                "law5_deterministic_reliability": result["scores"]["law5_deterministic_reliability"]
            },
            "pass_fail_status": status,
            "passed_threshold": passed,
            "threshold": threshold,
            "strictness": strictness
        },
        "violations": result.get("violations", []),
        "recommendations": result.get("recommendations", []),
        "strengths": result.get("strengths", []),
        "max_iterations": max_iterations,
        "message": f"{'✓ Validation passed!' if passed else '✗ Validation failed.'} Score: {overall_compliance:.1f}% (threshold: {threshold}%)"
    }
    
    return response


# Example 1: Validate passing text
print("Example 1: Text that passes validation")
print("=" * 80)

good_text = """
To solve this problem, I'll use a coordinated multi-protocol architecture:
1. ESL analyzes emotional context
2. REP handles logical reasoning with deductive inference
3. VVP validates rule structures before reasoning
4. Results validated against ground truth

This approach ensures deterministic outcomes through structured workflows
while maintaining computational efficiency through protocol specialization.
"""

result1 = iterative_validate_with_refinement(good_text, threshold=70.0, strictness="moderate")
print(f"Score: {result1['validation_result']['overall_compliance']:.1f}%")
print(f"Passed: {result1['validation_result']['passed_threshold']}")
print(f"Message: {result1['message']}")

# Example 2: Validate failing text with recommendations
print("\n" + "=" * 80)
print("\nExample 2: Text that needs refinement")
print("-" * 80)

poor_text = "Just use the biggest AI model and throw more compute at it."

result2 = iterative_validate_with_refinement(poor_text, threshold=70.0, strictness="moderate")
print(f"Score: {result2['validation_result']['overall_compliance']:.1f}%")
print(f"Passed: {result2['validation_result']['passed_threshold']}")
print(f"Message: {result2['message']}")
print(f"\nTop 3 Recommendations:")
for i, rec in enumerate(result2['recommendations'][:3], 1):
    print(f"  {i}. {rec}")

# Example 3: Simulated iterative workflow
print("\n" + "=" * 80)
print("\nExample 3: Simulated Iterative Refinement Workflow")
print("-" * 80)
print("(Demonstrating how AI client refines text across iterations)\n")

# Iteration 1
iter1_text = "Use AI for analysis"
print(f"Iteration 1: \"{iter1_text}\"")
iter1 = iterative_validate_with_refinement(iter1_text, threshold=75.0)
print(f"  Score: {iter1['validation_result']['overall_compliance']:.1f}% | Passed: {iter1['validation_result']['passed_threshold']}")

# Iteration 2 (simulated AI revision)
iter2_text = "Use coordinated protocol architecture with ESL for emotion analysis and REP for structured reasoning"
print(f"\nIteration 2: \"{iter2_text}\"")
iter2 = iterative_validate_with_refinement(iter2_text, threshold=75.0)
print(f"  Score: {iter2['validation_result']['overall_compliance']:.1f}% | Passed: {iter2['validation_result']['passed_threshold']}")

# Iteration 3 (further refinement)
iter3_text = "Implement coordinated multi-protocol architecture where ESL analyzes emotional context, REP performs deductive reasoning, and results are validated deterministically against ground truth for reliable outcomes"
print(f"\nIteration 3: \"{iter3_text[:70]}...\"")
iter3 = iterative_validate_with_refinement(iter3_text, threshold=75.0)
print(f"  Score: {iter3['validation_result']['overall_compliance']:.1f}% | Passed: {iter3['validation_result']['passed_threshold']}")

print(f"\n{'='*80}")
print("Summary: Score improved from {:.1f}% → {:.1f}% → {:.1f}% across iterations".format(
    iter1['validation_result']['overall_compliance'],
    iter2['validation_result']['overall_compliance'],
    iter3['validation_result']['overall_compliance']
))
print("Workflow shows how AI client uses validation feedback to refine responses")

## Example 9: Context-Aware Validation

Provide additional context to improve validation accuracy.

In [ ]:
text = "The system will learn from user interactions over time."

# Without context
result_basic = validator.validate(text, strictness="moderate")

# With context
context = {
    "domain": "machine_learning",
    "use_case": "adaptive_system",
    "governance_required": True
}
result_context = validator.validate(text, strictness="moderate", context=context)

print("Without Context:")
print(f"  Overall Compliance: {result_basic['scores']['overall_compliance']:.1f}%")
print(f"  Status: {result_basic['pass_fail_status']}\n")

print("With Context:")
print(f"  Overall Compliance: {result_context['scores']['overall_compliance']:.1f}%")
print(f"  Status: {result_context['pass_fail_status']}")

## Summary

This tutorial demonstrated SIM-ONE's Five Laws validation system:

### Key Capabilities
1. **Single text validation** - Validate individual AI responses against all Five Laws
2. **Batch validation** - Compare multiple responses and identify best performers
3. **Iterative refinement** - Track improvement across validation rounds
4. **Flexible strictness** - Adjust thresholds based on use case requirements

### When to Use Five Laws Validation
- **AI Response Quality Control** - Ensure responses meet governance standards
- **Model Comparison** - Evaluate different AI models against consistent criteria
- **Content Moderation** - Validate AI-generated content before publication
- **Compliance Auditing** - Document governance compliance for regulatory purposes

### API Reference
```python
from mcp_server.protocols.governance.five_laws_validator.* import *

validator = FiveLawsValidator()
result = validator.validate(
    text="Text to validate",
    strictness="moderate",  # "lenient" | "moderate" | "strict"
    context={"domain": "..."}  # optional
)
```

### Output Structure
```json
{
  "scores": {
    "law1_architectural_intelligence": 85.2,
    "law2_cognitive_governance": 78.5,
    "law3_truth_foundation": 92.1,
    "law4_energy_stewardship": 73.8,
    "law5_deterministic_reliability": 88.3,
    "overall_compliance": 83.6
  },
  "pass_fail_status": "PASS",
  "violations": [...],
  "recommendations": [...],
  "strengths": [...]
}
```

### Strictness Thresholds
| Level | Threshold | Use Case |
|-------|-----------|----------|
| Lenient | ≥60% | Exploratory content, drafts |
| Moderate | ≥70% | Standard production use |
| Strict | ≥85% | Critical governance systems |